## Install deepchem

### Colab

In [ ]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

In [ ]:
!pip install --pre deepchem-nightly==2.3.0
import deepchem
deepchem.__version__

In [ ]:
import tensorflow


device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

tensorflow.__version__

## Initialize

In [ ]:
import itertools
from dataclasses import dataclass
from sklearn.ensemble import RandomForestRegressor
from typing import Union, List, Tuple, NoReturn, Optional
import random
import os

import deepchem as dc
from deepchem.models import KerasModel
from tqdm import tqdm

ROOT_DIR = "/content/drive/My Drive/Colab Notebooks/deep learning for the life sciences/chapter_5_protein_binding"
DATA_DIR = os.path.join(ROOT_DIR, "data")

In [ ]:
%env DEEPCHEM_DATA_DIR=/content/drive/My Drive/Colab Notebooks/deep learning for the life sciences/chapter_5_protein_binding/data
%env

In [ ]:
import os
os.environ['DEEPCHEM_DATA_DIR']

## Load the data

### Cached data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path
import os

print(os.listdir(DATA_DIR))
print(os.listdir('/content/drive'))

In [ ]:
# for unmounting
# from google.colab import drive
# drive.flush_and_unmount()
